In [32]:
import pandas as pd

from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser

from ast import literal_eval

In [2]:
ls -lh data2

 Volume in drive C is Acer
 Volume Serial Number is 0823-6B7C

 Directory of C:\Python Scripts\MLChallenge\Wyzwanie_4


 Directory of C:\Python Scripts\MLChallenge\Wyzwanie_4\data2

04.09.2019  22:53    <DIR>          .
04.09.2019  22:53    <DIR>          ..
04.09.2019  22:53       133˙415˙034 job_ofer.csv
04.09.2019  22:54        39˙312˙744 job_ofer.csv.tar.gz
               2 File(s)    172˙727˙778 bytes
               2 Dir(s)  341˙131˙890˙688 bytes free


File Not Found


In [3]:
df = pd.read_csv('data2/job_ofer.csv')
df.shape

(36109, 8)

In [4]:
df.head()

,title,company_name,address,description,seniority_level,employment_type,job_function,industries
0,Machine Learning Engineer,Intellipro Group Inc,"Palo Alto, CA, US","['About The Company', ""W*** is reshaping the f...",Entry level,Full-time,Engineering,Information Technology and Services
1,Deep Learning Applied Researcher - Chicago,Ethosia,"Chicago, IL, US","['תיאור המשרה', 'Deep learning for Computer Vi...",Associate,Full-time,Other,Information Technology and Services
2,Machine Learning Engineer,Motorola Solutions,"Chicago, IL, US","['Company Overview', 'At Motorola Solutions, w...",Entry level,Full-time,Engineering,Information Technology and Services
3,Machine Learning / Data Scientist,Proprius LLC,"San Francisco, CA, US",['Our client is a digital invention agency foc...,Entry level,Full-time,Engineering,Information Technology and Services
4,Cloud Architect,TCS,"Framingham, Massachusetts, United States","['Technical/Functional Skills', ' ', 'Good to ...",Mid-Senior level,Full-time,Engineering,Information Technology and Services


## Word2Vec

In [6]:
corpus = df['title'].map( simple_preprocess)

In [7]:
model = Word2Vec(corpus, size=100, window=2, min_count=1)

C:\Users\PC\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [9]:
model.wv.most_similar('machine')

[('deep', 0.9580866694450378),
 ('acceleration', 0.8711884021759033),
 ('edge', 0.8700445890426636),
 ('inference', 0.8612096309661865),
 ('scientists', 0.8592015504837036),
 ('bentonville', 0.8566080331802368),
 ('big', 0.852530837059021),
 ('captivate', 0.8482269048690796),
 ('nlp', 0.846771240234375),
 ('computer', 0.8409063816070557)]

In [13]:
corpus = [
    ['machine', 'learning', 'c'],
    ['machine', 'learning', 'x'],
    ['y', 'machine', 'learning', 'w'],
    ['q', 'machine', 'learning', 'u', 'k'],
]
bigram = Phraser( Phrases(corpus, min_count=1, threshold=1))

bigram[ ['k', 'machine', 'learning', 'c']]

['k', 'machine_learning', 'c']

## Title + phrases

In [15]:
title_corpus = df['title'].map(simple_preprocess)

title_bigram = Phraser( Phrases(title_corpus, min_count=1, threshold=1))

In [17]:
title_bigram[ simple_preprocess('Deep Learning Applaied Resarcher - Chicago')]

['deep_learning', 'applaied', 'resarcher', 'chicago']

In [18]:
title_corpus_phrase = [title_bigram[sent] for sent in title_corpus]
model = Word2Vec(title_corpus_phrase, size=100, window=2, min_count=1)

C:\Users\PC\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [19]:
model.wv.most_similar('machine')

[('assistant_vice', 0.945888876914978),
 ('manager_domain', 0.9453252553939819),
 ('birmingham', 0.9451084733009338),
 ('llc', 0.9449663162231445),
 ('route', 0.9449475407600403),
 ('operator_truck', 0.9446684122085571),
 ('dry_van', 0.9434405565261841),
 ('engineer_iii', 0.9433798789978027),
 ('park', 0.9433251619338989),
 ('cdl_dedicated', 0.9433212280273438)]

In [20]:
def prepare_corpus(corpus, bigram):
    for sent in corpus:
        yield bigram[sent] + sent

In [21]:
simple_preprocess('Deep Learning Applaied Resarcher - Chicago')

['deep', 'learning', 'applaied', 'resarcher', 'chicago']

In [22]:
title_bigram[ ['deep', 'learning', 'appleid', 'resarcher', 'chicago']]

['deep_learning', 'appleid', 'resarcher', 'chicago']

In [25]:
ext_corp = list(prepare_corpus(title_corpus, title_bigram))
title_model = Word2Vec( ext_corp, size=100, window=2, min_count=1 )

In [26]:
title_model.wv.most_similar('machine')

[('deep', 0.9575784802436829),
 ('machine_learning', 0.952623724937439),
 ('learning', 0.9444158673286438),
 ('learning_product', 0.9353761672973633),
 ('artificial_intelligence', 0.9315321445465088),
 ('deep_learning', 0.930548369884491),
 ('nlp', 0.9288588762283325),
 ('computer_vision', 0.9255634546279907),
 ('nlp_research', 0.9244264364242554),
 ('big', 0.9222752451896667)]

## Description

In [28]:
descr_corpus = df['description'].map(simple_preprocess)
descr_bigram = Phraser(Phrases(descr_corpus, min_count=1, threshold=1))

In [29]:
ext_descr_corp = list(prepare_corpus(descr_corpus, descr_bigram))
descr_model = Word2Vec(ext_corp, size=100, window=2, min_count=1)

In [34]:
descr_model.most_similar('nlp')

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('alpharetta', 0.9918292164802551),
 ('computer', 0.988326907157898),
 ('big_data', 0.9869385361671448),
 ('artificial_intelligence', 0.9864896535873413),
 ('principal_data', 0.9858219623565674),
 ('computer_vision', 0.9855321049690247),
 ('deep', 0.9837415218353271),
 ('scientist_natural', 0.9834238886833191),
 ('catalyst', 0.9834015369415283),
 ('scientists', 0.9819791913032532)]

In [31]:
title_model.most_similar('nlp')

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('deep_learning', 0.9921468496322632),
 ('deep', 0.9903712272644043),
 ('artificial_intelligence', 0.9895111322402954),
 ('ml', 0.9889489412307739),
 ('computer', 0.9885504841804504),
 ('computer_vision', 0.9880896806716919),
 ('big_data', 0.9828800559043884),
 ('engineer_personalization', 0.9825940728187561),
 ('applied', 0.9825090169906616),
 ('junior_data', 0.9811336994171143)]

In [33]:
for line in df.sample()['description'].map(literal_eval).values[0]:
    print(line)
    print("")

Konica Minolta Business Solutions U.S.A., Inc., a global Fortune 1000 company and one of Forbes 2017 America’s Best Large Employers, is transforming the way we all do business.

From smart office technology and information management to cloud, mobility and IT services, we have a rich history of creating the products and services necessary to drive innovation and make work possible from anywhere at any time. Our award-winning products and solutions help companies around the world move information faster, improve quality and productivity, enhance security and facilitate the sharing of information. We are proud that our portfolio leads the industry while exceeding environmental standards.

At Konica Minolta, you'll work for an amazing technology company with growth opportunities, great benefits and talented, passionate co-workers.

Position Objective

Konica Minolta currently has an exciting opportunity for you to join us as a 

Major Account Executive!

Essential Job Functions

You will 